<table style="border: none" align="left">
    <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/pmservice/cars-4-you/master/static/images/logo.png" width="200" alt="Icon"></th>
       <th style="border: none"><font face="verdana" size="5" color="black"><b>Business area prediction and action recommendation</b></th>
   </tr>
</table>

<img align=left src="https://github.com/pmservice/cars-4-you/raw/master/static/images/action.png" width="550" alt="Icon">

Contents
- [0. Setup](#setup)
- [1. Introduction](#introduction)
- [2. Load and explore data](#load)
- [3. Create an Apache Spark machine learning model](#model)
- [4. Store the model in the Watson Machine Learning repository](#persistence)
- [5. Download the model](#download)
- [6. Deploy the model in the IBM Cloud](#persistence)
- [7. Score the model](#score)

**Note:** This notebook works correctly with kernel `Python 3.5 with Spark 2.1`.

<a id="setup"></a>
## 0. Setup

In this section please use below cell to upgrade the `watson-machine-learning-client`.

### Training data setup

Before you use the sample code in this notebook, you must perform the following setup tasks:

- Create a Watson Machine Learning Service instance (a free plan is offered).
- Create a Db2 Warehouse on Cloud Service instance (an entry plan is offered).
- Create the **CAR_RENTAL_TRAINING** in Db2 Warehouse on Cloud.
    - [Download data](https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/data/car_rental_training_data.csv) from git repository.
    - Click the Open the console to get started with Db2 Warehouse on Cloud icon.
    - Select the Load Data and Desktop load type.
    - Drag and drop previously downloaded file and press Next.
    - Select Schema to import data and click New Table.
    - Write name for new table than click Next to finish data import.
    - Use ; as field separator.
    - Click Next to create table with uploaded data.


### Package installation

In [1]:
!rm -rf $PIP_BUILD
!pip install --upgrade watson-machine-learning-client

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20181030044945-0000
    100% |################################| 942kB 791kB/s eta 0:00:01
    100% |################################| 51kB 1.9MB/s eta 0:00:01
    100% |################################| 61kB 2.0MB/s eta 0:00:01
    100% |################################| 122kB 2.7MB/s eta 0:00:01
    100% |################################| 51kB 1.7MB/s eta 0:00:01
    100% |################################| 51kB 1.5MB/s eta 0:00:01
    100% |################################| 153kB 2.5MB/s eta 0:00:01
    100% |################################| 8.7MB 104kB/s eta 0:00:01
    100% |################################| 143kB 2.6MB/s eta 0:00:01
    100% |################################| 61kB 2.0MB/s eta 0:00:01
    100% |################################| 788kB 931kB/s eta 0:00:01
    100% |################################| 194kB 2.2MB/s eta 0:00:01
    100% |################################| 13.8MB 68kB/s 

**Note**: Please restart the kernel (Kernel -> Restart)

<a id="introduction"></a>
## 1. Introduction

This notebook defines, trains and deploys the model that recommends specific Action for unstatisfied customers.

<a id="load"></a>
## 2. Load and explore data

In this section you will load the data as an Apache Spark DataFrame and perform a basic exploration.

Read data into Spark DataFrame from DB2 database and show sample record.

### Load data

**TIP:** Put your service credentials here. Just copy paste content of Credentials tab from service details (IBM Cloud)

In [2]:
db2_service_credentials = {
  "username": "***",
  "ssljdbcurl": "***",
  "host": "***",
  "https_url": "***",
  "dsn": "***",
  "hostname": "***",
  "jdbcurl": "***",
  "ssldsn": "***",
  "uri": "***",
  "password": "***"
}

In [3]:
# The code was removed by Watson Studio for sharing.

In [4]:
properties_db2 = {
    'driver': 'com.ibm.db2.jcc.DB2Driver',
    'jdbcurl': db2_service_credentials['jdbcurl'],
    'user': db2_service_credentials['username'],
    'password': db2_service_credentials['password']
}

In [5]:
from pyspark.sql import SparkSession
import json

spark = SparkSession.builder.getOrCreate()
table_name = 'CAR_RENTAL_TRAINING'
df_data = spark.read.jdbc(properties_db2['jdbcurl'], table='.'.join([properties_db2['user'], table_name]), properties=properties_db2)
df_data.head()

Row(ID=10, Gender='Male', Status='M', Children=1, Age=Decimal('46.000000'), Customer_Status='Inactive', Car_Owner='Yes', Customer_Service='They did not have the car I wanted.  upgraded me to a car I did not like and did not want.', Satisfaction=0, Business_Area='Product: Availability/Variety/Size', Action='Free Upgrade')

### Explore data

In [7]:
df_data.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Children: integer (nullable = true)
 |-- Age: decimal(14,6) (nullable = true)
 |-- Customer_Status: string (nullable = true)
 |-- Car_Owner: string (nullable = true)
 |-- Customer_Service: string (nullable = true)
 |-- Satisfaction: integer (nullable = true)
 |-- Business_Area: string (nullable = true)
 |-- Action: string (nullable = true)



**Tip:** Code above can be inserted using Data menu.  You have to select `Insert SparkSession DataFrame` option.

**Note:** Inserted code is modified to work with code in cells below.

As you can see, the data contains eleven fields. `Action` field is the one you would like to predict using feedback data in `Customer_Service` field.

In [8]:
print("Number of records: " + str(df_data.count()))

Number of records: 243


In [9]:
df_data.select('Business_area').groupBy('Business_area').count().show(truncate=False)

+----------------------------------+-----+
|Business_area                     |count|
+----------------------------------+-----+
|Service: Accessibility            |13   |
|Product: Functioning              |75   |
|Service: Attitude                 |12   |
|Service: Orders/Contracts         |16   |
|Product: Availability/Variety/Size|21   |
|Product: Pricing and Billing      |12   |
|Product: Information              |4    |
|Service: Knowledge                |90   |
+----------------------------------+-----+



In [10]:
df_data.select('Action').groupBy('Action').count().show(truncate=False)

+-------------------------+-----+
|Action                   |count|
+-------------------------+-----+
|NA                       |137  |
|Voucher                  |21   |
|Premium features         |15   |
|On-demand pickup location|28   |
|Free Upgrade             |42   |
+-------------------------+-----+



<a id="model"></a>
## 3. Create an Apache Spark machine learning model

In this section you will learn how to:

- [3.1 Prepare data for training a model](#prep)
- [3.2 Create an Apache Spark machine learning pipeline](#pipe)
- [3.3 Train a model](#train)

<a id="prep"></a>
### 3.1 Prepare data for training a model

In this subsection you will split your data into: train and test data set.

In [11]:
train_data, test_data = df_data.randomSplit([0.8, 0.2], 24)

print("Number of training records: " + str(train_data.count()))
print("Number of testing records : " + str(test_data.count()))

Number of training records: 200
Number of testing records : 43


### 3.2 Create the pipeline<a id="pipe"></a>

In this section you will create an Apache Spark machine learning pipeline and then train the model.

In [12]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler, HashingTF, IDF, Tokenizer, SQLTransformer
from pyspark.ml.classification import RandomForestClassifier, DecisionTreeClassifier, LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model, PipelineModel

In the following step, use the StringIndexer transformer to convert all the string fields to numeric ones.

In [13]:
tokenizer = Tokenizer(inputCol="Customer_Service", outputCol="words")

In [14]:
hashing_tf = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol='hash')
idf = IDF(inputCol=hashing_tf.getOutputCol(), outputCol="area_features", minDocFreq=5)

In [15]:
string_indexer_area = StringIndexer(inputCol="Business_Area", outputCol="area_label").fit(df_data)

In [16]:
dt_area = DecisionTreeClassifier(labelCol='area_label', featuresCol=idf.getOutputCol() , predictionCol='prediction_area', probabilityCol='probability_area', rawPredictionCol='rawPrediction_area')

Finally, convert the indexed labels back to original labels.

In [17]:
labelConverter = IndexToString(inputCol="prediction_area", outputCol="predictedAreaLabel", labels=string_indexer_area.labels)

In [18]:
string_indexer_gender = StringIndexer(inputCol="Gender", outputCol="gender_ix")
string_indexer_customer_status = StringIndexer(inputCol="Customer_Status", outputCol="customer_status_ix")
string_indexer_status = StringIndexer(inputCol="Status", outputCol="status_ix")
string_indexer_owner = StringIndexer(inputCol="Car_Owner", outputCol="owner_ix")

In [19]:
assembler = VectorAssembler(inputCols=["gender_ix", "customer_status_ix", "status_ix", "owner_ix", "Children", "Age", "Satisfaction", idf.getOutputCol()], outputCol="features")

In [20]:
string_indexer_action = StringIndexer(inputCol="Action", outputCol="label").fit(df_data)

In [21]:
label_action_converter = IndexToString(inputCol="prediction", outputCol="predictedActionLabel", labels=string_indexer_action.labels)

In [22]:
dt_action = DecisionTreeClassifier()

In [23]:
vector_assembler = VectorAssembler(inputCols=["gender_ix", "customer_status_ix", "status_ix", "owner_ix", "Children", "Age", "Satisfaction", 'prediction_area'], outputCol="features")

In [24]:
pipeline = Pipeline(stages=[tokenizer, hashing_tf, idf, string_indexer_area, dt_area, labelConverter, string_indexer_gender, string_indexer_customer_status, string_indexer_status, string_indexer_action, string_indexer_owner, vector_assembler, dt_action, label_action_converter])

### 3.3 Train the model

In [26]:
model = pipeline.fit(train_data)

### 3.4 Evaluate the model

In [27]:
predictions = model.transform(test_data)

In [28]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print("Accuracy = %g" % accuracy)

Accuracy = 0.744186


<a id="persistence"></a>
## 4. Store the model in the repository

In this section you will store trained model to Watson Machine Learning repository. When model is stored some metada is optional, however we provide it to be able to configure Continuous Learning System.

In [29]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

We need Watson Machine Learning credentials to be able to store model in repository.

**TIP:** Put watson Machine Learning service credentials here.

In [30]:
wml_credentials = {
  "apikey": "***",
  "iam_apikey_description": "***",
  "iam_apikey_name": "***",
  "iam_role_crn": "***",
  "iam_serviceid_crn": "***",
  "instance_id": "***",
  "password": "***",
  "url": "https://us-south.ml.cloud.ibm.com",
  "username": "***"
}

In [31]:
# The code was removed by Watson Studio for sharing.

In [32]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [33]:
client.version

'1.0.338'

### 4.2 Save the pipeline and model<a id="save"></a>

In [34]:
training_data_reference = {
 "name": "CARS4U training reference",
 "connection": db2_service_credentials,
 "source": {
  "tablename": table_name,
  "type": "dashdb"
 }
}

Define `output_data_schema` for the model

In [35]:
train_data_schema = train_data.schema
label_field = next(f for f in train_data_schema.fields if f.name == "Action")
label_field.metadata['values'] = string_indexer_action.labels

In [36]:
from pyspark.sql.types import *

input_fileds = filter(lambda f: f.name != "Action", train_data_schema.fields)

output_data_schema = StructType(list(input_fileds)). \
    add("prediction", DoubleType(), True, {'modeling_role': 'prediction'}). \
    add("predictedLabel", StringType(), True, {'modeling_role': 'decoded-target', 'values': string_indexer_action.labels}). \
    add("probability", ArrayType(DoubleType()), True, {'modeling_role': 'probability'})

Define model's metadata.

In [37]:
model_props = {
    client.repository.ModelMetaNames.NAME: "CARS4U - Action Recommendation Model",
    client.repository.ModelMetaNames.TRAINING_DATA_REFERENCE: training_data_reference,
    client.repository.ModelMetaNames.EVALUATION_METHOD: "multiclass",
    client.repository.ModelMetaNames.OUTPUT_DATA_SCHEMA: output_data_schema.jsonValue(),
    client.repository.ModelMetaNames.EVALUATION_METRICS: [
        {
           "name": "accuracy",
           "value": accuracy,
           "threshold": 0.7
        }
    ]
}

**Tip**: Use `client.repository.ModelMetaNames.show()` to get the list of available meta names.

Store the model.

In [38]:
published_model_details = client.repository.store_model(model=model, meta_props=model_props, training_data=train_data, pipeline=pipeline)

In [39]:
model_uid = client.repository.get_model_uid(published_model_details)
print(model_uid)

a757d335-38e3-4a55-8e67-2ced091fa9db


In [40]:
model_details = client.repository.get_model_details(model_uid)

<a id="download"></a>
## 5. Download the model

In [42]:
client.repository.download(model_uid)

In [43]:
from watson_machine_learning_client.utils import create_download_link

filename = 'downloaded_artifact.tar.gz'

In [44]:
create_download_link(filename)

<a id="deploy"></a>
## 6. Deploy model in the IBM Cloud

You can use following command to create online deployment in cloud.

In [45]:
deployment_details = client.deployments.create(model_uid=model_uid, name='CARS4U - Area and Action Model Deployment')



#######################################################################################

Synchronous deployment creation for uid: 'a757d335-38e3-4a55-8e67-2ced091fa9db' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='2e06ff73-f2cc-4483-9d60-8e31073efa45'
------------------------------------------------------------------------------------------------




You can use deployed model to score new data using scoring endpoint.

In [46]:
scoring_url = client.deployments.get_scoring_url(deployment_details)
print(scoring_url)

https://us-south.ml.cloud.ibm.com/v3/wml_instances/a73bf76d-a663-448a-b771-4f651f73d54e/deployments/2e06ff73-f2cc-4483-9d60-8e31073efa45/online


<a id="score"></a>
## 7. Score the model

In [47]:
fields = ['ID', 'Gender', 'Status', 'Children', 'Age', 'Customer_Status','Car_Owner', 'Customer_Service', 'Business_Area', 'Satisfaction']
values = [3785, 'Male', 'S', 1, 17, 'Inactive', 'Yes', 'The car should have been brought to us instead of us trying to find it in the lot.', 'Product: Information', 0]

In [61]:
payload_scoring = {"fields": fields,"values": [values]}
scoring_response = client.deployments.score(scoring_url, payload_scoring)

In [66]:
area_index = scoring_response['fields'].index('predictedAreaLabel')
action_index = scoring_response['fields'].index('predictedActionLabel')

print("Predicted Business Area: " + json.dumps(scoring_response['values'][0][area_index]))
print("Recommended action: " + json.dumps(scoring_response['values'][0][action_index]))

Predicted Business Area: "Product: Functioning"
Recommended action: "On-demand pickup location"


---

### Authors
Lukasz Cmielowski, PhD, is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.